In [ ]:
# загрузка библиотек
import pandas as pd
import requests as rq
import xml.etree.ElementTree as ET
from datetime import datetime
from bs4 import BeautifulSoup

### Задание 1

In [ ]:
# импорт, чистка данных
raw = pd.read_excel('test_input.xlsx', dtype=str)
# почему-то из экселя IE Code экспортируется без нуля в начале, даже если форсить формат строки
# не понимаю суть проблемы(
filename = raw.iloc[1].to_list()[1]
headers = raw.iloc[3].to_list()
df = raw.iloc[4:].reset_index(drop=True)
df.columns = headers
df['Issuance Date'] = list(map(lambda x: x[0:10],df['Issuance Date']))
df['SB Date'] = list(map(lambda x: x[0:10],df['SB Date']))
df['Client'] = list(map(lambda x: '"'+x+'"',df['Client']))
df['IE Code'] = list(map(lambda x: '0'*(10-len(x))+x if len(x)<10 else x ,df['IE Code']))
# приходится вручную добавлять нолики эхх

In [ ]:
# создание XML дерева
root = ET.Element('CERTDATA')
name = ET.SubElement(root, 'FILENAME')
name.text = filename
envelope = ET.SubElement(root, 'ENVELOPE')

In [ ]:
# внутренности конверта
tags = ['CERTNO','CERTDATE','STATUS','IEC','EXPNAME','BILLID','SDATE','SCC','SVALUE']
for i in range(0,len(df)):
    elem = df.iloc[i].to_list()
    ecert = ET.SubElement(envelope, 'ECERT')
    for j in range(0,len(elem)):
        item = ET.SubElement(ecert, tags[j])
        item.text = elem[j]

In [ ]:
# создание XML файла
tree = ET.ElementTree(root)
tree.write("output_xml.xml")

### Задание 2

In [ ]:
# парсим курс доллара
df2 = df.copy()
dates = list(map(lambda x: datetime.strptime(x,'%Y-%m-%d').strftime('%d.%m.%Y') ,df2['SB Date']))
USD = []
for i in dates:
    req = BeautifulSoup(rq.get('https://www.cbr.ru/currency_base/daily/?UniDbQuery.Posted=True&UniDbQuery.To='+i).text, 'html.parser')
    rates = list(map(lambda x: x.contents[0],req.findAll('td')))
    USD.append(rates[rates.index('Доллар США')+1].replace(',','.'))

In [ ]:
# добавляем цену в долларах
USDamount = []
for i in range(0,len(USD)):
    USDamount.append(str(round(float(df2['SB Amount'][i])/float(USD[i]),2)))
USDamount
df2['SB Amount USD'] = USDamount

In [ ]:
# создание XML дерева
root2 = ET.Element('CERTDATA')
name2 = ET.SubElement(root2, 'FILENAME')
name2.text = filename
envelope2 = ET.SubElement(root2, 'ENVELOPE')

In [ ]:
# внутренности конверта
tags = ['CERTNO','CERTDATE','STATUS','IEC','EXPNAME','BILLID','SDATE','SCC','SVALUE','SVALUEUSD']
for i in range(0,len(df2)):
    elem = df2.iloc[i].to_list()
    ecert = ET.SubElement(envelope2, 'ECERT')
    for j in range(0,len(elem)):
        item = ET.SubElement(ecert, tags[j])
        item.text = elem[j]

In [ ]:
# создание XML файла
tree2 = ET.ElementTree(root2)
tree2.write("output_xml_v2.xml")